In [35]:
import os
import shutil
import supervision as sv

# 데이터셋 로드
ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"../dataset/images",
    annotations_directory_path=f"../dataset/labels",
    data_yaml_path=f"../dataset/street-facilities/segment.yaml"
)

# 데이터셋 분리 (train 70%, val 15 % test 15%)
train_ds, val_test_ds = ds.split(split_ratio=0.7, random_state=1014, shuffle=True)
val_ds, test_ds = val_test_ds.split(split_ratio=0.5, random_state=535, shuffle=True)

print("train_ds length : ", len(train_ds))
print("val_ds length : ", len(val_ds))
print("test_ds length : ", len(test_ds))

train_ds length :  8429
val_ds length :  1806
test_ds length :  1807


### 분할된 데이터셋 클래스 분포 확인

In [36]:
import numpy as np

def class_counts(dataset : sv.DetectionDataset):
    n_classes = len(dataset.classes)
    count = np.zeros(n_classes, dtype=np.int16)
    for _, _, annotation in dataset.__iter__():
        # 사진 하나에 존재하는 0개 이상의 class object 들을 각각 센다
        for label in annotation.class_id:
            count[label] += 1

    return count

print("class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)")
for ds in [train_ds, val_ds, test_ds]:
    print(class_counts(ds))

class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)
[2706 6686 5514]
[ 507 1537 1247]
[ 535 1425 1216]


### 혹시 모를 버그 확인  
당연하지만 빈 라벨 파일에 대해서는 세지 않는다.

In [33]:
# 정상 보도블럭에 대한 라벨 파일. (바운딩 박스 x) 단지 빈 txt 파일이다.
empty_train_label = "../dataset/labels/2_09_0_1_4_1_20210917_0000516373.txt" 

list = [0, 0, 0]

with open(empty_train_label, 'r', encoding='utf-8') as f:
    for newline in f.readlines():
        list[newline] += 1

print(list)

[0, 0, 0]


In [37]:
# 저장할 디렉토리 경로 설정
train_images_dir = "../dataset/street-facilities/images/train"
train_labels_dir = "../dataset/street-facilities/labels/train"

val_images_dir = "../dataset/street-facilities/images/val"
val_labels_dir = "../dataset/street-facilities/labels/val"

test_images_dir = "../dataset/street-facilities/images/test"
test_labels_dir = "../dataset/street-facilities/labels/test"

# 디렉토리 생성 (존재하지 않는 경우에만)
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# DetectionDataset 객체의 이미지 및 라벨을 복사하여 저장
def save_dataset(dataset, images_dir_to_save, labels_dir_to_save):
    i = 1
    total = len(dataset)
    for image_path, image, annotation in dataset.__iter__():
        if i % 200 == 1 : print(f"{i} / {total}")
        # image_path : path/image_filename.jpg or .jpeg
        image_filename = os.path.basename(image_path) # 이미지 파일 이름
        label_filename = os.path.splitext(image_filename)[0] + ".txt" # 확장자명만 변경 (.jpg or .jpeg -> .txt)

        # label_path 조합
        label_path = "\\".join(image_path.split("\\")[:-2] + ["labels", label_filename])
        
        # 이미지 및 라벨을 각각 지정된 폴더로 복사
        shutil.copy(image_path, os.path.join(images_dir_to_save, image_filename))
        shutil.copy(label_path, os.path.join(labels_dir_to_save, label_filename))
        i += 1


''' 
    가끔 원래 디렉토리 외에 다른 디렉토리에'도' 저장되는 경우가 있는 듯 하다. 왜 그런지는 모르겠다.
    ex) .jpg 중 일부가 /labels 폴더에 저장되는 경우
'''

# train 데이터셋 저장
save_dataset(train_ds, train_images_dir, train_labels_dir)
# validation 데이터셋 저장
save_dataset(val_ds, val_images_dir, val_labels_dir)
# test 데이터셋 저장
save_dataset(test_ds, test_images_dir, test_labels_dir)

print("Train/Test 데이터셋 저장 완료.")

1 / 8429
201 / 8429
401 / 8429
601 / 8429
801 / 8429
1001 / 8429
1201 / 8429
1401 / 8429
1601 / 8429
1801 / 8429
2001 / 8429
2201 / 8429
2401 / 8429
2601 / 8429
2801 / 8429
3001 / 8429
3201 / 8429
3401 / 8429
3601 / 8429
3801 / 8429
4001 / 8429
4201 / 8429
4401 / 8429
4601 / 8429
4801 / 8429
5001 / 8429
5201 / 8429
5401 / 8429
5601 / 8429
5801 / 8429
6001 / 8429
6201 / 8429
6401 / 8429
6601 / 8429
6801 / 8429
7001 / 8429
7201 / 8429
7401 / 8429
7601 / 8429
7801 / 8429
8001 / 8429
8201 / 8429
8401 / 8429
1 / 1806
201 / 1806
401 / 1806
601 / 1806
801 / 1806
1001 / 1806
1201 / 1806
1401 / 1806
1601 / 1806
1801 / 1806
1 / 1807
201 / 1807
401 / 1807
601 / 1807
801 / 1807
1001 / 1807
1201 / 1807
1401 / 1807
1601 / 1807
1801 / 1807
Train/Test 데이터셋 저장 완료.


In [44]:
a = "..\\dataset\\street-facilities\\images\\3_13_1_1_2_1_20211029_0000927765.jpg"
b = a.split("\\")[:-1] + ["3_13_1_1_2_1_20211029_0000927765.txt"]
print(b)

print("\n", "\\".join(b))

['..', 'dataset', 'street-facilities', 'images', '3_13_1_1_2_1_20211029_0000927765.txt']

 ..\dataset\street-facilities\images\3_13_1_1_2_1_20211029_0000927765.txt
